In [ ]:
import config
import code_improvement.code_execution as execution

from utils import prompt, tools
from langchain.prompts import ChatPromptTemplate

In [2]:
def determine_term_list():
    term_prompt = ChatPromptTemplate.from_messages(
        [("system", "You are a very helpful assistant."), ("user", "{input}")]
    )
    term_chain = term_prompt | config.llm

    response = term_chain.invoke(
        {"input": prompt.concept_determination_prompt.format(config.user_input_IMU)}
    )
    term_list = response.content.split(", ")

    return term_list

In [3]:
def determine_URL_list(term_list):
    URL_list = []
    agent_executor = tools.create_agent(
        have_message_history=False, tool_list=[config.search], message_history=None
    )

    for term in term_list:
        print("Working on term: {} . . .".format(term))

        response = agent_executor.invoke(
            {
                "input": prompt.concept_searching_prompt.format(
                    config.user_input_IMU, term
                )
            }
        )

        URL_list.extend(response["output"].split(", "))

    return URL_list

In [4]:
def generate_high_level_design(agent_executor):
    response = agent_executor.invoke(
        {"input": prompt.high_level_design_prompt.format(config.user_input_IMU)},
        config=config.session_configuration,
    )

    return response["output"]

In [5]:
def generate_detailed_design(agent_executor):
    response = agent_executor.invoke(
        {"input": prompt.detailed_design_prompt.format(config.user_input_IMU)},
        config=config.session_configuration,
    )

    step_list = response["output"].split("\n\n")

    return step_list

In [6]:
def generate_code_segment(agent_executor, step_list):
    code_list = []

    for step_description in step_list:
        print(
            "Now, we are generating code segment for step {}".format(
                step_description.split("\n")[0]
            )
        )

        try:
            response = agent_executor.invoke(
                {
                    "input": prompt.code_segment_prompt.format(
                        config.user_input_IMU,
                        config.programming_language,
                        step_description.split("\n")[0],
                        step_description[step_description.index("\n") + 1 :],
                        config.programming_language,
                        step_description.split("\n")[0],
                    )
                },
                config=config.session_configuration,
            )
        except:
            continue

        code_segment = tools.sanitize_output(response["output"])
        code_list.append(code_segment)

    return code_list

In [7]:
def generate_final_code(agent_executor):
    response = agent_executor.invoke(
        {"input": prompt.code_combination_prompt.format(config.user_input_IMU)},
        config=config.session_configuration,
    )

    return tools.sanitize_output(response["output"])

In [8]:
def execute_code(code, data_path):
    with open("generated_code.py", "w") as file:
        file.write(code)

    command = "python3 generated_code.py -i {}".format(data_path)
    status, program_output = execution.run(command, True)

    return status, program_output

In [9]:
# first determine a list of terms to be serached
term_list = determine_term_list()
print(
    "According to your requirements, we decide to search for the definition of the following terms:",
    term_list,
)

According to your requirements, we decide to search for the definition of the following terms: ['Wireless Sensor Data Mining', 'WISDM dataset', 'Accelerometer data', 'human activity recognition', 'PyTorch framework']


In [10]:
URL_list = determine_URL_list(term_list)
print("Here is the URL list we crawled:")
for URL in URL_list:
    print(URL)

c:\Users\zjusl\AppData\Local\Programs\Python\Python310\lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Working on term: WIreless Sensor Data Mining . . .


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'WIreless Sensor Data Mining definition'}`


We have searched the following information:
url: https://journals.sagepub.com/doi/10.1155/2013/406316
content: Data mining in sensor networks is the process of extracting application-oriented models and patterns with acceptable accuracy from a continuous,
url: https://www.cis.fordham.edu/wisdm/
content: The WISDM (Wireless Sensor Data Mining) Lab is concerned with collecting the sensor data from smart phones and other modern mobile devices.
url: https://www.sciencedirect.com/science/article/abs/pii/B978012408091100004X
content: This article concentrates on two different aspects on Wireless Sensor Networks (WSNs): Data Mining (DM) in WSNs and enhancements of DM using appropriate tools,
url: https://www.researchgate.net/publication/258393184_Data_Mining_Techniques_for_Wireless_Sensor_Networks_A_Surv

In [11]:
document_list = tools.load_new_webpage(URL_list)
agent_with_chat_history = tools.create_agent(
    have_message_history=True,
    tool_list=[config.search, tools.create_context_retrieval_tool(document_list)],
    message_history=config.message_history,
)

We will further search information from these websites:
https://www.cis.fordham.edu/wisdm/
https://www.kaggle.com/datasets/die9origephit/human-activity-recognition
https://archive.ics.uci.edu/ml/machine-learning-databases/00507/WISDM-dataset-description.pdf
https://www.cis.fordham.edu/wisdm/dataset.php
https://en.wikipedia.org/wiki/Accelerometer
https://indatalabs.com/blog/human-activity-recognition
https://www.labellerr.com/blog/human-activity-recognition/
https://www.sciencedirect.com/topics/computer-science/human-activity-recognition
https://www.v7labs.com/blog/human-activity-recognition
https://en.wikipedia.org/wiki/PyTorch
https://pytorch.org/


c:\Users\zjusl\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
c:\Users\zjusl\AppData\Local\Programs\Python\Python310\lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
high_level_design = generate_high_level_design(agent_with_chat_history)
print("We first provide the algorithm design outline:\n{}".format(high_level_design))



> Entering new AgentExecutor chain...

Invoking: `context_document_search` with `{'query': 'WISDM dataset human activity recognition PyTorch'}`






Human Activity Recognition | Kaggle





























































Human Activity Recognition (HAR): Fundamentals, Models, Datasets




























ProductsResourcesCompanySign upLog inBlogWebinarAcademyResourcesBook a demoBook a demoBook a demoComputer visionHuman Activity Recognition (HAR): Fundamentals, Models, Datasets
            20
             min read—Mar 27, 2023Dive into the state-of-the-art of Human Activity Recognition (HAR) and discover real-life applications plus datasets to try out.Deval ShahGuest AuthorHuman Activity Recognition (HAR) is an exciting research area in computer vision and human-computer interaction.Automatic detection of human physical activity has become crucial in pervasive computing, interpersonal communication, and human behavior analysis.The broad usage of HAR 

In [13]:
detailed_design_step_list = generate_detailed_design(agent_with_chat_history)

print("Based on the high level design, we provide the detailed design as:")
for element in detailed_design_step_list:
    print(element, "\n")



> Entering new AgentExecutor chain...

Invoking: `context_document_search` with `{'query': 'WISDM dataset preprocessing steps'}`




  


 WISDM Lab: Wireless Sensor Data Mining










WISDM: WIreless Sensor Data Mining
Menu




Home
About

Overview
Publications
Funding
Equipment
News


Datasets

Resources

Dev Tools
Related Work
Related Applications
Conferences


Members





Wireless SensorData Mining


Wireless Sensor Data Mining






Next Meeting06/25/2014, 12:00 pmWISDM Lab 
Click the button below to register for our mailing list
Sign Up!

 
The WISDM (Wireless Sensor Data Mining) Lab is concerned with collecting the sensor data from smart phones and other modern 
            mobile devices (e.g., tablet computers, music players, etc.) and mining this sensor data for useful knowledge. Currently our efforts are 
            mainly focused on the acclerometer and GPS sensor data from these devices, but in the future we will mine the audio sensors (microphones), 
            im

In [14]:
code_segment_list = generate_code_segment(
    agent_with_chat_history, detailed_design_step_list
)

Now, we are generating code segment for step Step 2: Dataset Preparation


> Entering new AgentExecutor chain...

Invoking: `context_document_search` with `{'query': 'WISDM dataset preprocessing and dataset preparation'}`




  


 WISDM Lab: Wireless Sensor Data Mining










WISDM: WIreless Sensor Data Mining
Menu




Home
About

Overview
Publications
Funding
Equipment
News


Datasets

Resources

Dev Tools
Related Work
Related Applications
Conferences


Members





Wireless SensorData Mining


Wireless Sensor Data Mining






Next Meeting06/25/2014, 12:00 pmWISDM Lab 
Click the button below to register for our mailing list
Sign Up!

 
The WISDM (Wireless Sensor Data Mining) Lab is concerned with collecting the sensor data from smart phones and other modern 
            mobile devices (e.g., tablet computers, music players, etc.) and mining this sensor data for useful knowledge. Currently our efforts are 
            mainly focused on the acclerometer and GPS sensor data from the

In [15]:
final_code = generate_final_code(agent_with_chat_history)



> Entering new AgentExecutor chain...
```python
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import argparse

class ActivityRecognitionCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(ActivityRecognitionCNN, self).__init__()

        # Define a sequential container for the layers
        self.cnn_layers = nn.Sequential(
            nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),

            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),

            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=

In [28]:
dataset_path = input("Please input the path to your dataset: ")
# dataset_path = "/home/iot/Documents/data/IMU/WISDM/WISDM_ar_v1.1_raw.txt"

In [ ]:
print("Next, we will proceed to the supervised code generation and modification stage.")

for epoch in range(config.epoch_number):
    status, program_output = execute_code(final_code, dataset_path)
    while status != 0:
        response = agent_with_chat_history.invoke(
            {
                "input": prompt.correct_grammar_prompt.format(
                    config.user_input_IMU, program_output
                )
            },
            config=config.session_configuration,
        )
        final_code = tools.sanitize_output(response["output"])

        status, program_output = execute_code(final_code, dataset_path)

    response = agent_with_chat_history.invoke(
        {
            "input": prompt.improve_code_prompt.format(
                config.user_input_IMU, program_output
            )
        },
        config=config.session_configuration,
    )
    high_level_design = response["output"]
    detailed_design_step_list = generate_detailed_design(agent_with_chat_history)
    code_segment_list = generate_code_segment(
        agent_with_chat_history, detailed_design_step_list
    )
    final_code = generate_final_code(agent_with_chat_history)

Next, we will proceed to the supervised code generation and modification stage.
************** START **************
Traceback (most recent call last):
File "/home/iot/Documents/AutoNLP/generated_code.py", line 226, in <module>
main(args.input)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 190, in main
features = extract_features(segments)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 101, in extract_features
return np.array(features)
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (8578, 15) + inhomogeneous part.
************** FAILED **************


> Entering new AgentExecutor chain...
```python
# HAR_system.py
import pandas as pd
import numpy as np
from scipy import stats, signal
from scipy.fftpack import fft
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
i

In [ ]:
# search for all the generated comprehensive code
combined_code_list = []

for index in range(0, len(config.message_history.messages), 2):
    question = config.message_history.messages[index].content

    if (
        "Based on the user's problem and all the generated code segments for each module, please constructively integrate them to form a comprehensive and cohesive piece of code. Please also embed detailed comments."
        in question
    ):
        combined_code = tools.sanitize_output(
            config.message_history.messages[index + 1].content
        )
        combined_code_list.append(combined_code)

In [74]:
program_output_list = []

for index in range(len(combined_code_list)):
    status, program_output = execute_code(combined_code_list[index], dataset_path)
    program_output_list.append(program_output)

************** START **************
Traceback (most recent call last):
File "/home/iot/Documents/AutoNLP/generated_code.py", line 227, in <module>
main(args.input)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 191, in main
features = extract_features(segments)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 102, in extract_features
return np.array(features)
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (8578, 15) + inhomogeneous part.
************** FAILED **************
************** START **************
/home/iot/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunct

In [ ]:
running_result = ""

for index in range(len(program_output_list)):
    running_result = "{}\n\n<version_{}>\n{}\n</version_{}>\n\n".format(
        running_result, index + 1, program_output_list[index], index + 1
    )


response = agent_with_chat_history.invoke(
    {"input": prompt.final_decision_prompt.format(running_result)},
    config=config.session_configuration,
)



> Entering new AgentExecutor chain...
2

> Finished chain.


In [ ]:
version_index = int(response["output"])
final_code = combined_code_list[version_index - 1]

print("Here is the final version of code to solve your problem.")
print("============================================")
print(final_code)
print("============================================")

Here is the final version of code to solve your problem.

# HAR_system.py
import pandas as pd
import numpy as np
from scipy import stats, signal
from scipy.fftpack import fft
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import copy
import argparse


# Define the human activity recognition model using PyTorch
class HARModel(nn.Module):
    def __init__(
        self,
        input_size,
        output_size,
        conv_channels,
        lstm_hidden_size,
        lstm_layers,
        dropout_prob,
    ):
        super(HARModel, self).__init__()
        self.conv_layers = nn.ModuleList()
        input_channels = 1
        for output_channels in conv_channels:
            self.conv_layers.append(
                nn.Sequential(
                    nn.Conv

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": prompt.user_documentation_prompt.format(
            config.user_input_IMU, final_code
        )
    },
    config=config.session_configuration,
)



> Entering new AgentExecutor chain...
# User Documentation for the Human Activity Recognition System

## Introduction

### User's Problem
The user aims to develop a human activity recognition (HAR) system using the Wireless Sensor Data Mining (WISDM) dataset. This dataset includes accelerometer data collected from smartphones, which can be used to identify human activities. The task involves preprocessing the WISDM data, dividing it into training and testing parts, constructing a machine learning model for activity recognition, and finally evaluating the model's average recognition accuracy on the test dataset.

### The Code
The provided code is a Python script that implements a HAR system using the PyTorch framework. The script is designed to run on systems with a powerful GPU and uses a combination of convolutional neural networks (CNNs), long short-term memory (LSTM) networks, and attention mechanisms to classify accelerometer data into activity categories.

## Getting Started

##